In [3]:
exported_file = '/home/ec2-user/SageMaker/serperi/system/sessions/scal/One/data/exported_data_2022-10-12_14-08.csv'
lines = open(exported_file,'r').read().splitlines()[1:]
lines[:2]

['https://proquest.com/docview/1151636504,rel,1.000',
 'https://proquest.com/docview/1151316407,rel,1.000']

In [46]:
import re
import pandas as pd
ids = list(map(lambda line: re.findall('docview/([0-9]{10})', line)[0], lines))


scores = list(map(lambda line: float(line.split(',')[-1]), lines))

from utils.tdmstudio import get_filename, get_title_and_text, get_date, get_title
# texts = list(map(lambda id_: get_title_and_text(get_filename(id_)) , ids))
# titles = list(map(lambda id_: get_title(get_filename(id_)) , ids))

# dates = list(map(lambda id_: get_date(get_filename(id_)) , ids))

df = pd.DataFrame({'id':ids,'score':scores})
df = df.sort_values(by=['score'], ascending=False).iloc[12:,]
df

,id,score
9028,1143705076,0.088
11169,1140875377,0.086
13855,2606146691,0.084
6961,1237561744,0.082
16034,1412235848,0.082
...,...,...
1269,1313801276,0.044
11274,1136258477,0.044
1267,1241357671,0.044
11280,1238581584,0.044


In [34]:
import numpy as np 

rand = np.random.default_rng(2022)

upper_range = [3,
               7,
               19,
               49,
               130,
               346,
               915,
               2425,
               6423,
               17014,
               ]
sizes = [3, 7, 10, 10, 10, 10, 10, 10, 10, -1]
selections=[]
for upper,size in zip(upper_range, sizes):
    aux = df.iloc[:upper]
    if size!=-1:
        selections.append(np.random.choice(aux['id'], size=size, replace=False))
    else:
        size = 100 - len(set([elem for list_ in selections for elem in list_]))
        selections.append(np.random.choice(aux['id'], size=size, replace=False))
        

In [45]:
count=0
texts=[]
titles=[]
dates=[]
all_ids=[]
batch_numbers = []
for batch, list_ in enumerate(selections):
    ids = list_
    all_ids+=list(ids)
    texts += list(map(lambda id_: get_title_and_text(get_filename(id_)) , ids))
    titles += list(map(lambda id_: get_title(get_filename(id_)) , ids))

    dates += list(map(lambda id_: get_date(get_filename(id_)) , ids))
    batch_numbers+=[batch+1]*len(list_)

export = pd.DataFrame({'batch':batch_numbers, 'id':all_ids, 'text':texts, 'dates':dates, 'title':titles})
export

,batch,id,text,dates,title
0,1,1143705076,COMING HOME.\n \n \n\n \n\...,1994-07-02,COMING HOME.
1,1,1140875377,Has diversity gone too far?.\n \n ...,1997-03-15,Has diversity gone too far?.
2,1,2606146691,Multiculturalism doesn't divide. It encourages...,2018-09-14,Multiculturalism doesn't divide. It encourages...
3,2,1237561744,Multiculturalism a policy of fusion or fission...,1984-09-08,Multiculturalism a policy of fusion or fission.
4,2,2606146691,Multiculturalism doesn't divide. It encourages...,2018-09-14,Multiculturalism doesn't divide. It encourages...
...,...,...,...,...,...
106,10,1239736266,Hammers at PM's credibility.\n \n ...,1974-06-07,Hammers at PM's credibility.
107,10,1143636694,Separatist road show kicks off on Prairies.\n ...,1998-03-03,Separatist road show kicks off on Prairies.
108,10,1366051544,City to be named a cultural capital of Canada....,2005-01-27,City to be named a cultural capital of Canada.
109,10,1313809416,This is no straying academic. The man's a natu...,1974-05-25,This is no straying academic. The man's a natu...


In [48]:
export.to_csv('/home/ec2-user/SageMaker/serperi/system/sessions/scal/One/data/random_sample_to_evaluate_suggestions.csv')

In [49]:
freq = {}
for i in range(len(export)):
    id_= export['id'].iloc[i]
    if id_ not in freq:
        freq[id_]=1
    else:
        freq[id_]+=1
[id_ for id_ in freq if freq[id_]>1]

['1143705076',
 '1140875377',
 '2606146691',
 '1237561744',
 '1412235848',
 '1239427076',
 '1146095326',
 '1240032290',
 '1142941709']